In [ ]:
# !pip install --upgrade efficientnet-pytorch
!pip install --upgrade timm

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/nishika/Japanese Painting Classification"

In [19]:
# ライブラリ読み込み
import glob
import random
import numpy as np
import os
import pandas as pd
import collections
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
# from efficientnet_pytorch import EfficientNet
import timm
from timm.data import rand_augment_transform

In [ ]:
SEED = 43
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(SEED)
current_device = torch.cuda.current_device()
print("Device:", torch.cuda.get_device_name(current_device))

In [21]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TRAIN_DIR = "./data/train/"
TEST_DIR = "./data/test/"
MODELS_PATH = "./models/"
TRAIN_BATCH_SIZE = 20
VALID_BATCH_SIZE = 8
NUM_CLASSES = 8
USE_PRETRAINED = True
EPOCHS = 30
N_SPLITS = 5

In [22]:
# データのオーギュメンテーションと前処理
class ImageTransform():
    def __init__(self):
        self.data_transform = {
            "train": transforms.Compose([
                transforms.Resize(256),  # リサイズ
                transforms.CenterCrop(256),  # 画像中央を切り取り
                transforms.RandomHorizontalFlip(),  # 半分の確率で画像を左右反転させる
                rand_augment_transform("rand-m15-n2-w0", hparams=None),
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
            ]),
            "test": transforms.Compose([
                transforms.Resize(256),  # リサイズ
                transforms.CenterCrop(256),  # 画像中央を切り取り
                transforms.RandomHorizontalFlip(),  # 半分の確率で画像を左右反転させる
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 標準化
        ])
    }

    def __call__(self, image, phase="trian"):
        return self.data_transform[phase](image)

In [23]:
preview = False
if preview:
    image_file_path = './data/test/n_00000.jpg'
    img = Image.open(image_file_path)
    plt.imshow(img)
    plt.show()

    transform = ImageTransform()

    test_transformed = transform(img, phase='test')
    test_transformed = test_transformed.numpy().transpose((1, 2, 0))
    test_transformed = np.clip(test_transformed, 0, 1)
    plt.imshow(test_transformed)
    plt.show()

    train_transformed = transform(img, phase='train')
    train_transformed = train_transformed.numpy().transpose((1, 2, 0))
    train_transformed = np.clip(train_transformed, 0, 1)
    plt.imshow(train_transformed)
    plt.show()

In [24]:
def make_df_train_folded(n_splits=5):

    train_df = pd.read_csv("./data/train.csv")
    train_df["kfold"] = -1
    gender_status = train_df["gender_status"].tolist()

    skfold = StratifiedKFold(n_splits, shuffle=False)
    for fold, (_, valid_indexes) in enumerate(skfold.split(range(len(gender_status)), gender_status)):
        for i in valid_indexes:
            train_df.iat[i,2] = fold
    return train_df

In [25]:
# Dataset作成
class KaokoreDataset(data.Dataset):
    def __init__(self, df_data, root_dir, transform=None, phase='train', return_name=False):
        self.root_dir = root_dir
        self.image_list = df_data["image"].tolist()
        self.label_list = df_data["gender_status"].tolist()
        self.transform = transform
        self.phase = phase
        self.return_name = return_name
        
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, index):
        image_path = self.image_list[index]
        image = Image.open(self.root_dir + image_path)
        image = self.transform(image, self.phase)

        label = self.label_list[index]
        if self.return_name:
            return image, label, image_path
        else:
            return image, label

In [26]:
class Classifier(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        
        self.model = timm.create_model("tf_efficientnet_b7_ns", USE_PRETRAINED, num_classes=num_classes)
        self.model = self.model.to(DEVICE)

    def forward(self, inputs):

        outputs = self.model(inputs)

        return outputs

    def return_params(self):
        layer_params = []
        fc_params = []
        head_params = []
        fc_param_names = ["classifier.weight", "classifier.bias"]
        head_param_names = ["conv_head.weight", "bn2.weight", "bn2.bias"]
        for name, param in self.model.named_parameters():
            if name in fc_param_names:
                param.requires_grad = True
                fc_params.append(param)
                if "weight" in name:
                    nn.init.xavier_normal_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)
            elif name in head_param_names:
                param.requires_grad = True
                head_params.append(param)
            else:
                param.requires_grad = True
                layer_params.append(param)

        return layer_params, head_params, fc_params

In [27]:
def train_fn(dataloader, model, criterion, optimizer, device, epoch, scheduler=None):

    model.train()
    total_loss = 0
    total_corrects = 0
    all_labels = []
    all_preds = []

    progress = tqdm(dataloader, total=len(dataloader))

    for i, (inputs, labels) in enumerate(progress):
        progress.set_description(f"<Train> Epoch{epoch+1}")

        optimizer.zero_grad()

        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        del inputs
        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        del outputs

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        del loss
        total_corrects += torch.sum(preds == labels)

        all_labels += labels.tolist()
        all_preds += preds.tolist()
        del labels, preds

        progress.set_postfix(loss=total_loss/(i+1), f1=f1_score(all_labels, all_preds, average="micro"))

    train_loss = total_loss / len(dataloader)
    train_acc = total_corrects.double().cpu().detach().numpy() / len(dataloader.dataset)
    train_f1 = f1_score(all_labels, all_preds, average="micro")

    return train_loss, train_acc, train_f1

In [28]:
def eval_fn(dataloader, model, criterion, device, epoch):
    model.eval()
    total_loss = 0
    total_corrects = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        progress = tqdm(dataloader, total=len(dataloader))
        
        for i, (inputs, labels) in enumerate(progress):
            progress.set_description(f"<Valid> Epoch{epoch+1}")

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            del inputs
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            del outputs

            total_loss += loss.item()
            del loss
            total_corrects += torch.sum(preds == labels)

            all_labels += labels.tolist()
            all_preds += preds.tolist()
            del labels, preds

            progress.set_postfix(loss=total_loss/(i+1), f1=f1_score(all_labels, all_preds, average="micro"))

    valid_loss = total_loss / len(dataloader)
    valid_acc = total_corrects.double().cpu().detach().numpy() / len(dataloader.dataset)

    valid_f1 = f1_score(all_labels, all_preds, average="micro")

    return valid_loss, valid_acc, valid_f1

In [29]:
def plot_training(train_losses, train_accs, train_f1s,
                  valid_losses, valid_accs, valid_f1s,
                  epoch, fold):
    
    loss_df = pd.DataFrame({"Train":train_losses,
                            "Valid":valid_losses},
                        index=range(1, epoch+2))
    loss_ax = sns.lineplot(data=loss_df).get_figure()
    loss_ax.savefig(f"./figures/loss_plot_fold={fold}.png", dpi=300)
    loss_ax.clf()

    acc_df = pd.DataFrame({"Train":train_accs,
                           "Valid":valid_accs},
                          index=range(1, epoch+2))
    acc_ax = sns.lineplot(data=acc_df).get_figure()
    acc_ax.savefig(f"./figures/acc_plot_fold={fold}.png", dpi=300)
    acc_ax.clf()

    f1_df = pd.DataFrame({"Train":train_f1s,
                          "Valid":valid_f1s},
                         index=range(1, epoch+2))
    f1_ax = sns.lineplot(data=f1_df).get_figure()
    f1_ax.savefig(f"./figures/f1_plot_fold={fold}.png", dpi=300)
    f1_ax.clf()

In [30]:
def trainer(fold, df_data):

    df_train = df_data[df_data.kfold != fold].reset_index(drop=True)
    df_valid = df_data[df_data.kfold == fold].reset_index(drop=True)

    train_dataset = KaokoreDataset(df_train, TRAIN_DIR, ImageTransform(), phase="train")
    valid_dataset = KaokoreDataset(df_valid, TRAIN_DIR, ImageTransform(), phase="test")

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=4,
        shuffle=True
    )
    valid_dataloader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=2,
        shuffle=False
    )

    model = Classifier(NUM_CLASSES)
    layer_params, head_params, fc_params = model.return_params()

    train_criterion = nn.CrossEntropyLoss()
    valid_criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD([
        {"params": layer_params, "lr": 1e-2, "weight_decay": 1e-4, "momentum": 0.9},
        {"params": head_params, "lr": 1e-2, "weight_decay": 1e-4, "momentum": 0.9},
        {"params": fc_params, "lr": 1e-1, "weight_decay": 1e-4, "momentum": 0.9}
    ])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=len(train_dataloader)*20, gamma=0.5)

    train_losses = []
    train_accs = []
    train_f1s = []
    valid_losses = []
    valid_accs = []
    valid_f1s = []

    best_loss = np.inf
    best_acc = 0
    best_f1 = 0

    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_fn(train_dataloader, model, train_criterion, optimizer, DEVICE, epoch, scheduler)
        valid_loss, valid_acc, valid_f1 = eval_fn(valid_dataloader, model, valid_criterion, DEVICE, epoch)
        print(f"Loss: {valid_loss}  Acc: {valid_acc}  f1: {valid_f1}  ", end="")

        train_losses.append(train_loss)
        train_accs.append(train_acc)
        train_f1s.append(train_f1)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
        valid_f1s.append(valid_f1)

        plot_training(train_losses, train_accs, train_f1s,
                      valid_losses, valid_accs, valid_f1s,
                      epoch, fold)
        
        best_loss = valid_loss if valid_loss < best_loss else best_loss
        besl_acc = valid_acc if valid_acc > best_acc else best_acc
        if valid_f1 > best_f1:
            best_f1 = valid_f1
            print("model saving!", end="")
            torch.save(model.model.state_dict(), MODELS_PATH + f"best_model_{fold}.pt")
        print("\n")

    return best_f1

In [ ]:
df_data = make_df_train_folded(N_SPLITS)

In [ ]:
f1_scores = []
for fold in range(N_SPLITS):
    print(f"fold {fold}", "="*80)
    f1 = trainer(fold, df_data)
    f1_scores.append(f1)
    print(f"<fold={fold}> best score: {f1}\n")

In [ ]:
cv = sum(f1_scores) / len(f1_scores)
print(f"CV: {cv}")

In [ ]:
lines = ""
for i, f1 in enumerate(f1_scores):
    line = f"fold={i}: {f1}\n"
    lines += line
lines += f"CV    : {cv}"
with open("./models/result.txt", mode='w') as f:
    f.write(lines)

In [ ]:
models = []
for fold in range(N_SPLITS):
    model = Classifier()
    model.model.load_state_dict(torch.load(MODELS_PATH + f"best_model_{fold}.pt"))
    model.eval()
    models.append(model)

In [ ]:
df_test = pd.read_csv("./data/sample_submission.csv")
test_dataset = KaokoreDataset(df_test, TEST_DIR, ImageTransform(), phase="test")
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size = 64, shuffle=False
)

In [ ]:
with torch.no_grad():
    progress = tqdm(test_dataloader, total=len(test_dataloader))
    final_output = []

    for inputs, _ in progress:
        progress.set_description("<Test>")

        inputs = inputs.to(DEVICE)

        outputs = []
        for model in models:
            output = model(inputs)
            outputs.append(output)

        outputs = sum(outputs) / len(outputs)
        outputs = torch.softmax(outputs, dim=1).cpu().detach().tolist()
        outputs = np.argmax(outputs, axis=1)

        final_output.extend(outputs)

In [ ]:
sample = pd.read_csv("./data/sample_submission.csv")
sample.loc[:, "gender_status"] = final_output
try:
    sample.to_csv("./outputs/submission_cv{}.csv".format(str(cv).replace(".", "")), index=False)
except NameError:
     sample.to_csv("./outputs/submission.csv", index=False)
sample.head()

In [ ]:
def print_classification_report(df_data, trained_model=""):

    all_preds = []
    all_labels = []
    all_probs = []

    for fold in range(N_SPLITS):

        df_valid = df_data[df_data.kfold == fold].reset_index(drop=True)
        valid_dataset = KaokoreDataset(df_valid, TRAIN_DIR, ImageTransform(), phase="test")
        valid_dataloader = torch.utils.data.DataLoader(
            valid_dataset,
            batch_size=VALID_BATCH_SIZE,
            num_workers=4,
            shuffle=False
        )

        model = Classifier()
        model.model.load_state_dict(torch.load(MODELS_PATH + trained_model + f"best_model_{fold}.pt"))
        model.eval()

        with torch.no_grad():

            progress = tqdm(valid_dataloader, total=len(valid_dataloader))
            final_output = []

            for inputs, labels in progress:

                inputs = inputs.to(DEVICE)

                outputs = model(inputs)

                outputs = torch.softmax(outputs, dim=1).cpu().detach().tolist()
                preds = np.argmax(outputs, axis=1)

                all_preds += preds.tolist()
                all_labels += labels.tolist()

    cr = classification_report(all_labels, all_preds)
    freq = collections.Counter(all_labels)
    freq = [freq[i] for i in range(8)]
    cm = confusion_matrix(all_labels, all_preds)
    cm = cm / freq
    print(cr)
    sns.heatmap(cm, cmap="Reds", annot=True)
    plt.show()

In [ ]:
print_classification_report(df_data)